In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "ti"
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s" % (source_language, target_language)

In [3]:
from google.colab import files
uploaded = files.upload()

Saving en_ti.csv to en_ti.csv


In [0]:
import pandas as pd
import io
data = pd.read_csv(io.BytesIO(uploaded['en_ti.csv']))
# Dataset is now stored in a Pandas Dataframe

In [6]:
data.head()

,English,Tigrigna
0,17 Jehovah now said to Moses : 2 “ Speak to th...,17 የሆዋ ድማ ንሙሴ ኸምዚ ኢሉ ተዛረቦ ፦ 2 “ ንእስራኤላውያን ተዛረቦ...
1,"6 So Moses spoke to the Israelites , and all t...","6 ሙሴ ድማ ንእስራኤላውያን ተዛረቦም , ኵሎም ሹማምንቲ ኸኣ ኣባትር ማ..."
2,"8 On the next day , when Moses went into the t...",8 ንጽብሒቱ ሙሴ ናብ ድንኳን ምስክር ምስ ኣተወ እንሆ በትሪ ኣሮን ...
3,10 Jehovah then said to Moses : “ Put Aaron ’ ...,10 የሆዋ ድማ ንሙሴ “ በትሪ ኣሮን ንደቂ ዕልወት ምልክት ክትኰኖም ም...
4,12 The Israelites then said to Moses : “ Now w...,12 እስራኤላውያን ድማ ንሙሴ “ ሕጂ ሞትና ጠፋእና ኵላትና ጠፋእና ...


In [0]:
data = data.rename(columns={"English":"source_sentence", "Tigrigna":"target_sentence"})


In [8]:
data.head()

,source_sentence,target_sentence
0,17 Jehovah now said to Moses : 2 “ Speak to th...,17 የሆዋ ድማ ንሙሴ ኸምዚ ኢሉ ተዛረቦ ፦ 2 “ ንእስራኤላውያን ተዛረቦ...
1,"6 So Moses spoke to the Israelites , and all t...","6 ሙሴ ድማ ንእስራኤላውያን ተዛረቦም , ኵሎም ሹማምንቲ ኸኣ ኣባትር ማ..."
2,"8 On the next day , when Moses went into the t...",8 ንጽብሒቱ ሙሴ ናብ ድንኳን ምስክር ምስ ኣተወ እንሆ በትሪ ኣሮን ...
3,10 Jehovah then said to Moses : “ Put Aaron ’ ...,10 የሆዋ ድማ ንሙሴ “ በትሪ ኣሮን ንደቂ ዕልወት ምልክት ክትኰኖም ም...
4,12 The Israelites then said to Moses : “ Now w...,12 እስራኤላውያን ድማ ንሙሴ “ ሕጂ ሞትና ጠፋእና ኵላትና ጠፋእና ...


In [0]:
# Do the split between dev/test/train and create parallel corpora
num_dev_patterns = 1000
num_test_patterns = 1000
df = data
# Lower case the corpora
df["source_sentence"] = df["source_sentence"].str.lower()
df["target_sentence"] = df["target_sentence"].str.lower()


devtest = df.tail(num_dev_patterns + num_test_patterns)
test = devtest.tail(num_test_patterns)
dev = devtest.head(num_dev_patterns)
stripped = df.drop(df.tail(num_dev_patterns + num_test_patterns).index)

stripped[["source_sentence"]].to_csv("train.en", index=False)
stripped[["target_sentence"]].to_csv("train.ti", index=False)

dev[["source_sentence"]].to_csv("dev.en", index=False)
dev[["target_sentence"]].to_csv("dev.ti", index=False)

test[["source_sentence"]].to_csv("test.en", index=False)
test[["target_sentence"]].to_csv("test.ti", index=False)

In [10]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 2051 (delta 25), reused 22 (delta 10), pack-reused 2006
Receiving objects: 100% (2051/2051), 2.39 MiB | 16.10 MiB/s, done.
Resolving deltas: 100% (1414/1414), done.
Processing /content/joeynmt
     |████████████████████████████████| 266kB 9.8MB/s 
     |████████████████████████████████| 307kB 46.8MB/s 
     |████████████████████████████████| 215kB 41.9MB/s 
     |████████████████████████████████| 51kB 23.9MB/s 
     |████████████████████████████████| 51kB 22.6MB/s 
     |████████████████████████████████| 737kB 48.1MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=69430 sha256=44cf14657322cee0a283c96264e4a7ddaff77daa6da79354f4c0229e0734e8b1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ur4sp2w1/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for p

In [11]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
! mkdir joeynmt/data/
! mkdir joeynmt/data/enti/
! export data_path=joeynmt/data/$src$tgt/
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
# Create the /data/enti/ folder explicityly
! cp train.* joeynmt/data/enti/ 
! cp test.* joeynmt/data/enti/
! cp dev.* joeynmt/data/enti/
! cp bpe.codes.4000 $data_path
! ls $data_path

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Tigrigna Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/enti/vocab.txt

cp: missing destination file operand after 'bpe.codes.4000'
Try 'cp --help' for more information.
bpe.codes.4000	dev.ti	   sample_data	test.ti       train.ti
dev.bpe.en	drive	   test.bpe.en	train.bpe.en  vocab.en
dev.bpe.ti	en_ti.csv  test.bpe.ti	train.bpe.ti  vocab.ti
dev.en		joeynmt    test.en	train.en
BPE Tigrigna Sentences
"25 ደቂ ጋ@@ ድን ደቂ ሮ@@ ቤ@@ ልን ከኣ ንሙሴ ኸምዚ በልዎ ፦ “ ገላዉ@@ ኻ ኸም@@ ቲ ጐይታይ ዝኣዘዞ ኺ@@ ገብሩ እዮም . 26 ቈ@@ ልዑ@@ ናን ኣንስ@@ ት@@ ናን ማል@@ ናን ኵሉ እንስ@@ ሳታት ዘ@@ ቤ@@ ት@@ ናን ኣብኡ ኣብ ከተማታት ጊልዓድ ይኹ@@ ኑ , 27 ገላዉ@@ ኻ ግና ከምቲ ጐይታይ ዝ@@ በሎ ንው@@ ግእ እተ@@ ዓ@@ ጥ@@ ቁ ዘ@@ በሉ ዅሎም ኣብ ቅድሚ የሆዋ ኺ@@ ዋግ@@ ኡ ኺ@@ ሳገ@@ ሩ እዮም . ”"
28 ሙሴ ድማ ን@@ ኻህን ኣል@@ ኣ@@ ዛ@@ ርን ንእ@@ ያሱ ወዲ ነ@@ ዌ@@ ን ን@@ ርእስ@@ ታት ማይ ቤ@@ ታት ነገ@@ ዳት እስራኤልን ብዛዕባ@@ ኦም ኣዘ@@ ዘ . 29 ሙሴ ኸኣ ከምዚ በሎም ፦ “ ደቂ ጋ@@ ድን ደቂ ሮ@@ ቤ@@ ልን ኵላቶም ኣብ ቅድሚ የሆዋ ንው@@ ግእ ተዓ@@ ጢ@@ ቖም ምሳኻትኩም ዮርዳኖስ እንተ ተ@@ ሳ@@ ጊ@@ ሮም እታ ምድሪ ድማ ኣብ ቅድሜ@@ ኹም እንተ ተገ@@ ዚ@@ ኣ ሽዑ ምድሪ ጊልዓድ ርስቲ ጌርኩም ሃብ@@ ዎም . 30 ኣ@@ ጽዋ@@ ር ኣልዒ@@ ሎም ምሳኻትኩም እንተ ዘይ@@ ተ@@ ሳ@@ ጊ@@ ሮም ግና ኣብ ምድሪ ከነ@@ ኣን ኣብ መንጎ@@ ኹም ይስ@@ ፈ@@ ሩ .

In [12]:

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.ti  train.bpe.en	train.ti
dev.bpe.en	dev.ti	     test.en	  train.bpe.ti
dev.bpe.ti	test.bpe.en  test.ti	  train.en


In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)
name = '%s%s' % (source_language, target_language)

config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "models/{name}_transformer/12000.ckpt" # if given, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "noam"            # Try switching from plateau to Noam scheduling
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    patience: 8
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0002
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30 # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 400 # Decrease this for testing
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8
        embeddings:
            embedding_dim: 512
            scale: True
            dropout: 0.
        # typically ff_size = 4 x hidden_size
        hidden_size: 512
        ff_size: 2048
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8
        embeddings:
            embedding_dim: 512
            scale: True
            dropout: 0.
        # typically ff_size = 4 x hidden_size
        hidden_size: 512
        ff_size: 2048
        dropout: 0.3
""".format(name=name, source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [14]:
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2019-10-13 12:52:42,472 Hello! This is Joey-NMT.
2019-10-13 12:52:44,000 Total params: 46427648
2019-10-13 12:52:44,001 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [15]:
! cat joeynmt/models/enti_transformer/validations.txt

Steps: 400	Loss: 183767.51562	PPL: 183.82838	bleu: 0.00000	LR: 0.00027951	*
Steps: 800	Loss: 166880.53125	PPL: 113.84876	bleu: 1.18889	LR: 0.00055902	*
Steps: 1200	Loss: 149437.20312	PPL: 69.40469	bleu: 2.57752	LR: 0.00063789	*
Steps: 1600	Loss: 140869.54688	PPL: 54.42731	bleu: 3.85995	LR: 0.00055243	*
Steps: 2000	Loss: 136958.40625	PPL: 48.71057	bleu: 5.12391	LR: 0.00049411	*
Steps: 2400	Loss: 136141.56250	PPL: 47.59462	bleu: 5.49313	LR: 0.00045105	*
Steps: 2800	Loss: 136970.31250	PPL: 48.72703	bleu: 5.70431	LR: 0.00041760	
Steps: 3200	Loss: 140427.06250	PPL: 53.74828	bleu: 5.76537	LR: 0.00039063	
Steps: 3600	Loss: 144299.90625	PPL: 59.99102	bleu: 6.35670	LR: 0.00036828	
Steps: 4000	Loss: 147836.78125	PPL: 66.32362	bleu: 5.96325	LR: 0.00034939	
Steps: 4400	Loss: 150693.10938	PPL: 71.92242	bleu: 6.70454	LR: 0.00033313	


In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!mkdir "$gdrive_path/models/"
!cp -r joeynmt/models/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [20]:
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 400	Loss: 183767.51562	PPL: 183.82838	bleu: 0.00000	LR: 0.00027951	*
Steps: 800	Loss: 166880.53125	PPL: 113.84876	bleu: 1.18889	LR: 0.00055902	*
Steps: 1200	Loss: 149437.20312	PPL: 69.40469	bleu: 2.57752	LR: 0.00063789	*
Steps: 1600	Loss: 140869.54688	PPL: 54.42731	bleu: 3.85995	LR: 0.00055243	*
Steps: 2000	Loss: 136958.40625	PPL: 48.71057	bleu: 5.12391	LR: 0.00049411	*
Steps: 2400	Loss: 136141.56250	PPL: 47.59462	bleu: 5.49313	LR: 0.00045105	*
Steps: 2800	Loss: 136970.31250	PPL: 48.72703	bleu: 5.70431	LR: 0.00041760	
Steps: 3200	Loss: 140427.06250	PPL: 53.74828	bleu: 5.76537	LR: 0.00039063	
Steps: 3600	Loss: 144299.90625	PPL: 59.99102	bleu: 6.35670	LR: 0.00036828	
Steps: 4000	Loss: 147836.78125	PPL: 66.32362	bleu: 5.96325	LR: 0.00034939	
Steps: 4400	Loss: 150693.10938	PPL: 71.92242	bleu: 6.70454	LR: 0.00033313	


In [22]:
! cd joeynmt; python3 -m joeynmt test models/enti_transformer/config.yaml


2019-10-13 15:10:10,118 -  dev bleu:   5.86 [Beam search decoding with beam size = 5 and alpha = 1.0]
2019-10-13 15:14:18,063 - test bleu:   5.31 [Beam search decoding with beam size = 5 and alpha = 1.0]
